In [1]:
from pyspark.sql.functions import lit, col
import json
from cdk.services.api.energi_data_service import EnergiDataService
from cdk.common_modules.access.secrets import Secrets
from cdk.common_modules.utility.spark_config import SparkConfig
from cdk.common_modules.utility.spark_session_builder import SparkSessionBuilder

storage_account_name = "adlsthuehomelakehousedev"

# Set Spark configurations
spark_config = SparkConfig(Secrets())

# Add jars to install
spark_config.add_jars_to_install(['hudi', 'azure_storage'])

# Add storage account access
spark_config.add_storage_account_access(storage_account_name, method='access_key')

# Build SparkSession
spark = SparkSessionBuilder("ViewAzureData", "spark://spark-master:7077", spark_config).build()


INFO:root:Loading secret ADLS_adlsthuehomelakehousedev_access_key


:: loading settings :: url = jar:file:/opt/bitnami/python/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
org.apache.hadoop#hadoop-azure added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9305cf4b-180a-4df4-9bc6-a48718da6f96;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 in central
	found org.apache.hadoop#hadoop-azure;3.3.3 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.fasterxml.jackson.core#jackson-core;2.13.2 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#fai

23/09/24 20:26:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
dataset_name = "ConsumptionDK3619codehour"
dataset_path = "energi_data_service"
container_name = "landing"

# landing_path = f"hdfs://namenode:9000/data/landing/energi_data_service/{dataset_name}"
landing_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{dataset_path}"


In [3]:
# pyspark
df = spark. \
  read. \
  format("hudi"). \
  load(landing_path)

23/09/24 20:27:28 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/09/24 20:27:28 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file


In [4]:
df.show(10)

+-------------------+--------------------+--------------------+----------------------+--------------------+---------------+--------+--------------------+--------+--------------------+-------------------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|Consumption_MWh|DK19Code|           DK19Title|DK36Code|           DK36Title|             HourDK|            HourUTC|
+-------------------+--------------------+--------------------+----------------------+--------------------+---------------+--------+--------------------+--------+--------------------+-------------------+-------------------+
|  20230924073511918|20230924073511918...|HourUTC:2023-09-0...|                      |d06cf81e-b1df-44d...|      204.35082|       A|Landbrug, skovbru...|       A|Landbrug, skovbru...|2023-09-02T20:00:00|2023-09-02T18:00:00|
|  20230924073511918|20230924073511918...|HourUTC:2023-09-0...|                      |d06cf81e-b1df-44d.

In [7]:
print(df.count())
df.select('DK19Code','DK36Code','HourDK').filter((col('DK19Code')=='C') & (col('DK36Code')=='CI') ).distinct().orderBy('HourDK').show(30)

816


+--------+--------+-------------------+
|DK19Code|DK36Code|             HourDK|
+--------+--------+-------------------+
|       C|      CI|2023-09-02T00:00:00|
|       C|      CI|2023-09-02T01:00:00|
|       C|      CI|2023-09-02T02:00:00|
|       C|      CI|2023-09-02T03:00:00|
|       C|      CI|2023-09-02T04:00:00|
|       C|      CI|2023-09-02T05:00:00|
|       C|      CI|2023-09-02T06:00:00|
|       C|      CI|2023-09-02T07:00:00|
|       C|      CI|2023-09-02T08:00:00|
|       C|      CI|2023-09-02T09:00:00|
|       C|      CI|2023-09-02T10:00:00|
|       C|      CI|2023-09-02T11:00:00|
|       C|      CI|2023-09-02T12:00:00|
|       C|      CI|2023-09-02T13:00:00|
|       C|      CI|2023-09-02T14:00:00|
|       C|      CI|2023-09-02T15:00:00|
|       C|      CI|2023-09-02T16:00:00|
|       C|      CI|2023-09-02T17:00:00|
|       C|      CI|2023-09-02T18:00:00|
|       C|      CI|2023-09-02T19:00:00|
|       C|      CI|2023-09-02T20:00:00|
|       C|      CI|2023-09-02T21:00:00|


In [4]:
# Group by DK19Code, DK36Code to count number of rows
df.groupBy('DK19Code','DK36Code').count().orderBy('DK19Code','DK36Code').show(30)

+--------+--------+-----+
|DK19Code|DK36Code|count|
+--------+--------+-----+
|       A|       A|  360|
|       B|       B|  360|
|       C|      CA|  360|
|       C|      CC|  360|
|       C|CE_CF_CD|  360|
|       C|      CG|  360|
|       C|      CH|  360|
|       C|      CI|  360|
|       C|      CJ|  360|
|       C|      CK|  360|
|       C|      CL|  360|
|       C|   CM_CB|  360|
|       D|       D|  360|
|       E|       E|  360|
|       F|       F|  360|
|       G|       G|  360|
|       H|       H|  360|
|       I|       I|  360|
|       J|      JA|  360|
|       J|   JB_JC|  360|
|       K|       K|  360|
|       L|       L|  360|
|       M|      MA|  360|
|       M|      MB|  360|
|       M|      MC|  360|
|       N|       N|  360|
|       O|       O|  360|
|       P|       P|  360|
|      PR|      PR|  360|
|       Q|      QA|  360|
+--------+--------+-----+
only showing top 30 rows



In [2]:
from cdk.common_modules.delta.delta_state import DeltaState

storage_account_name = "adlsthuehomelakehousedev"
container_name = "landing"
dataset_name = "ConsumptionDK3619codehour"


# Create DeltaState object
delta_state = DeltaState(spark, storage_account_name, container_name, dataset_name)

# Set delta state
# delta_state.set_delta_state("2023-09-03T00:00")

In [4]:
delta_state.get_delta_state(default_value="2023-01-01T00:00")

23/09/24 16:43:04 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/09/24 16:43:04 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file
23/09/24 16:43:17 WARN HoodieBackedTableMetadata: Metadata table was not found at path abfss://utility@adlsthuehomelakehousedev.dfs.core.windows.net/delta/delta_table/.hoodie/metadata


'2023-01-01T00:00'

In [6]:
delta_path = f"abfss://utility@{storage_account_name}.dfs.core.windows.net/delta/delta_table"

df = spark.read.format("hudi").load(delta_path).createOrReplaceTempView("hudi_delta_snapshot")

spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_delta_snapshot order by commitTime").show()


+-----------------+
|       commitTime|
+-----------------+
|20230924164310010|
+-----------------+



In [8]:
# Read hudi metadata from delta table
df = spark.read.format("hudi").load(delta_path+'/.hoodie/metadata')
df.show()

23/09/24 16:49:15 WARN HoodieFileIndex: No partition columns available from hoodie.properties. Partition pruning will not work


+------------------+----+--------------------+-------------------+-------------------+
|               key|type|  filesystemMetadata|BloomFilterMetadata|ColumnStatsMetadata|
+------------------+----+--------------------+-------------------+-------------------+
|__all_partitions__|   1|   {. -> {0, false}}|               null|               null|
|                 .|   2|{927fee05-73c9-47...|               null|               null|
+------------------+----+--------------------+-------------------+-------------------+

